# 整体PipeLine

## 1. RAG存储与索引构建

### 1.1 存储

In [1]:
import json

# 定义一个函数来读取JSON文件并转换其中的对象为字符串列表
def read_json_objects(file_path):
    # 打开文件并加载JSON数据
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # 将每个对象转换为字符串并添加到列表中
    objects_as_strings = [json.dumps(obj, ensure_ascii=False) for obj in data]
    return objects_as_strings

# 使用示例
file_path = 'test_result.json'  # 你的JSON文件路径
objects_as_strings = read_json_objects(file_path)
print(objects_as_strings)
print(objects_as_strings[0])
len(objects_as_strings)


['{"spu_id": 640, "name": "足银9999描金海棠花银手镯开口实心纯银手镯古风新中式520礼物", "keyword": "手镯、闭口手镯、足银9999", "introduction": "新中式风格，送女朋友", "description": "<p>手镯手镯手镯</p>", "category_id": 84, "brand_id": 4, "pic_url": "https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/151d5ce2e46d364effcebd407e7eb95075e2d289516f722e7b9b3946cfef53d8.png", "slider_pic_urls": "[\\"https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/8e9d9e96520f21e25b4ac730bc5e6c62e1d36f0e26c31a87685c18bce58c4498.jpg\\",\\"https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/06e8b168cab5b48336ca29253c8e6cecad5e0ad8baacb2a98a9a5365136b9804.jpg\\",\\"https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/3dbf47a546bc3665953519a2611c17daf2f3192ab3b16fd62ace20b993521c42.jpg\\",\\"https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/d8656c5219a89a57d1d88e2838d7a48aa451f7eb993acb29ffcc79a5b377ef6c.jpg\\",\\"https://yinshi-1309445959.cos.ap-beijing.myqcl

19

In [2]:
# 使用 OpenAI Embedding
# from langchain.embeddings.openai import OpenAIEmbeddings
# 使用百度千帆 Embedding
# from langchain.embeddings.baidu_qianfan_endpoint import QianfanEmbeddingsEndpoint
# 使用我们自己封装的智谱 Embedding，需要将封装代码下载到本地使用
from zhipuai_embedding import ZhipuAIEmbeddings
import os
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。
# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())
# 定义 Embeddings
# embedding = OpenAIEmbeddings()
embedding = ZhipuAIEmbeddings()
# embedding = QianfanEmbeddingsEndpoint()

# 定义持久化路径
persist_directory = './data_base/chroma'

In [3]:
!rm -rf './data_base/chroma'

In [ ]:
from langchain.vectorstores.chroma import Chroma

vectordb = Chroma.from_texts(
    texts=objects_as_strings, 
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
vectordb.persist()
print(f"向量库中存储的数量：{vectordb._collection.count()}")

### 1.2 检索

In [1]:
from dotenv import load_dotenv, find_dotenv
import os
from zhipuai_embedding import ZhipuAIEmbeddings
from langchain.vectorstores.chroma import Chroma


_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# 定义 Embeddings
embedding = ZhipuAIEmbeddings()

# 向量数据库持久化路径
persist_directory = './data_base/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)
print(f"向量库中存储的数量：{vectordb._collection.count()}")

# Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted

向量库中存储的数量：19


In [2]:
# 普通检索
question="女朋友带哪个？"
sim_text = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_text)}")
for i, sim_doc in enumerate(sim_text):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content}", end="\n--------------\n")

检索到的内容数：3
检索到的第0个内容: 
{"spu_id": 651, "name": "足银999银手镯女捕梦网手镯手环银镯子高级感送闺蜜送女朋友19克", "keyword": "捕梦网", "introduction": "捕梦网", "description": "<p>详情</p>", "category_id": 84, "brand_id": 4, "pic_url": "https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/f29eb9854efe0822d30a54d6dda4f6f4a54cf65f22565fca24e25de75165036a.jpg", "slider_pic_urls": "[\"https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/6b23262332274e42de99413a83601a4be7e8ea9c8c0d02081ad97fbfee2c677e.jpg\",\"https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/d4cf40c593cad9895a060d093a53c639ced04ece1a9cffab673d2987f601cc43.jpg\",\"https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/f29eb9854efe0822d30a54d6dda4f6f4a54cf65f22565fca24e25de75165036a.jpg\"]", "sort": 0, "status": 1, "spec_type": 0, "price": 25900, "market_price": 22000, "cost_price": 24000, "stock": 74, "delivery_types": "2", "delivery_template_id": null, "give_integral": 0, "sub_commission

MMR检索
如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。
最大边际相关性 (MMR, Maximum marginal relevance) 可以帮助我们在保持相关性的同时，增加内容的丰富度。
核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。


In [8]:
mmr_text = vectordb.max_marginal_relevance_search(question,k=3)
for i, sim_doc in enumerate(mmr_text):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content}", end="\n--------------\n")

Number of requested results 20 is greater than number of elements in index 19, updating n_results = 19


MMR 检索到的第0个内容: 
{"spu_id": 651, "name": "足银999银手镯女捕梦网手镯手环银镯子高级感送闺蜜送女朋友19克", "keyword": "捕梦网", "introduction": "捕梦网", "description": "<p>详情</p>", "category_id": 84, "brand_id": 4, "pic_url": "https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/f29eb9854efe0822d30a54d6dda4f6f4a54cf65f22565fca24e25de75165036a.jpg", "slider_pic_urls": "[\"https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/6b23262332274e42de99413a83601a4be7e8ea9c8c0d02081ad97fbfee2c677e.jpg\",\"https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/d4cf40c593cad9895a060d093a53c639ced04ece1a9cffab673d2987f601cc43.jpg\",\"https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/f29eb9854efe0822d30a54d6dda4f6f4a54cf65f22565fca24e25de75165036a.jpg\"]", "sort": 0, "status": 1, "spec_type": 0, "price": 25900, "market_price": 22000, "cost_price": 24000, "stock": 74, "delivery_types": "2", "delivery_template_id": null, "give_integral": 0, "sub_commission_type"

### 1.3 生成

In [9]:
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.vectorstores.chroma import Chroma
# 定义 Embeddings
embedding = ZhipuAIEmbeddings()


# 定义持久化路径
persist_directory = './data_base/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：19


In [11]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import os

from langchain_community.chat_models import ChatZhipuAI
ZHIPUAI_API_KEY = os.environ["ZHIPUAI_API_KEY"]
llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)
llm.invoke("请你自我介绍一下自己！")

AIMessage(content='你好，我是一个人工智能助手，基于清华大学 KEG 实验室与智谱 AI 于 2023 年共同训练的语言模型 GLM2 开发而成。我的任务是针对用户的问题和要求提供适当的答复和支持。我可以通过分析大量文本数据来学习和理解自然语言，从而与人类进行有效的沟通。请随时向我提出问题或请求帮助，我会尽我所能为您提供服务。', response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 10, 'total_tokens': 92}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-87b70a23-91a5-468e-a442-82c9553fd1c3-0')

In [3]:
from langchain_core.tracers import langchain
from langchain.prompts import PromptTemplate

template = """你是一个杰出的优秀银饰客服，称呼其他人为姐姐或者小哥哥，根据，礼貌热情地进行推荐和回答。
使用以下上下文作为你可能推荐的饰品，放在<context></context> XML标签内。
<context>
{context}
</context>
当回答用户时：
你应该根据给出的内容进行回答，主要目的是推荐<context></context>中的银饰产品，给出关于该饰品有价值的回答，尽量促成用户购买。
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)

question_1 = "推荐几款适合女朋友带的银饰"
an = QA_CHAIN_PROMPT.invoke({"question":question_1,"context":"1. 【足银999.9古法莫比乌斯扭纹纯银手镯开口小众款】这款手镯的设计灵感来源于莫比乌斯环，象征着无限的爱与永恒，开口设计方便佩戴，非常适合追求独特个性的女朋友。价格：27900元"})

# from langchain.chains import RetrievalQA
# langchain.debug = True
# qa_chain = RetrievalQA.from_chain_type(llm,
#                                        retriever=vectordb.as_retriever(),
#                                        return_source_documents=True,
#                                        chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


# from langchain_core.prompts import PromptTemplate

# prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

# prompt_template.invoke({"topic": "cats"})
print(an)

text='你是一个杰出的优秀银饰客服，称呼其他人为姐姐或者小哥哥，根据，礼貌热情地进行推荐和回答。\n使用以下上下文作为你可能推荐的饰品，放在<context></context>\xa0XML标签内。\n<context>\n1. 【足银999.9古法莫比乌斯扭纹纯银手镯开口小众款】这款手镯的设计灵感来源于莫比乌斯环，象征着无限的爱与永恒，开口设计方便佩戴，非常适合追求独特个性的女朋友。价格：27900元\n</context>\n当回答用户时：\n你应该根据给出的内容进行回答，主要目的是推荐<context></context>中的银饰产品，给出关于该饰品有价值的回答，尽量促成用户购买。\n问题: 推荐几款适合女朋友带的银饰\n'


In [13]:
question_1 = "推荐几款适合女朋友带的银饰"
question_2 = "中秋节想给母亲买个银饰有什么推荐嘛？"

result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])


h:\exploit_python\environment\anaconda\envs\sml\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


大模型+知识库后回答 question_1 的结果：
亲爱的姐姐/小哥哥，您好呀！给女朋友挑选银饰的话，我们这里有几款特别受欢迎的纯银手镯，不仅设计独特，而且每款都有其独特的寓意，非常适合作为礼物哦：

1. 【足银999.9古法莫比乌斯扭纹纯银手镯开口小众款】
   - 这款手镯的设计灵感来源于莫比乌斯环，象征着无限的爱与永恒，开口设计方便佩戴，非常适合追求独特个性的女朋友。
   - 价格：27900元
   - ![足银999.9古法莫比乌斯扭纹纯银手镯开口小众款](https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/5cd34aabc2416065ae9a67502f453fc69f5d34d0a9324f9d583d073e5d5d2b7e.jpg)

2. 【足银999.9实心唐草纹描金纯银手镯闭口女款】
   - 唐草纹是中国传统图案，寓意着生机勃勃，描金工艺让手镯更显高贵气质，闭口设计适合不喜欢开口款式的女朋友。
   - 价格：32900元
   - ![足银999.9实心唐草纹描金纯银手镯闭口女款](https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/c0ee05b5ede9e4bfcea9eb3aee19c90b68428135a27e3c71274e684bfd94079e.jpg)

3. 【足银999.9古法珐琅彩兰花纯银手镯开口小众款】
   - 珐琅彩兰花设计，不仅美观大方，而且兰花象征着高洁和爱情，是表达爱意的绝佳选择。
   - 价格：36900元
   - ![足银999.9古法珐琅彩兰花纯银手镯开口小众款](https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/b1177d19112d909ac2725bbad6f212ecfe3eeffe7bb4d1892536e4138f946941.jpg)

4. 【足银9999描金海棠花银手镯开口实心纯银手镯古风新中式】
   - 海棠花寓意着美丽和爱情，古风新中式的风格适合喜欢传统文化氛围的女朋友。
   - 价

In [15]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
text_1 = llm.invoke(prompt_template)

prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
text_2 = llm.invoke(prompt_template)

print(text_1)
print(text_2)

content='适合女朋友的银饰有很多种，根据不同的风格和场合，以下是一些建议：\n\n1. 简约银项链\n   - 简约的银项链适合各种场合，既可以搭配日常装扮，也可以搭配晚宴服装。可以选择一个细小的银链条，搭配一个小小的吊坠，如心形、星星或月亮等。\n\n2. 银手链\n   - 银手链是一个时尚且实用的选择。可以挑选一个带有装饰性元素的银手链，如小珠子、小十字架或心形挂饰。\n\n   - 流苏银手链也很受欢迎，它们具有优雅的动感。\n\n3. 银耳环\n   - 如果女朋友喜欢戴耳环，可以选择一对简约的银耳钉或小巧的银圈耳环。流苏耳环或几何形状的耳环也是不错的选择。\n\n   - 小巧的钻石或宝石镶嵌在银耳环上，也能增添一份优雅与华丽。\n\n4. 银戒指\n   - 一个简约的银戒指可以作为一个日常配饰。可以选择一个平滑的银圈戒指或带有小装饰的戒指，如小钻石、珍珠或宝石。\n\n5. 定制银饰\n   - 如果你想要更加个性化，可以考虑定制银饰。可以根据女朋友的喜好或两人的共同回忆，定制一个独一无二的银饰，如刻上特殊日期、名字或两人的生肖等。\n\n在选择银饰时，注意以下几点：\n- 确保银饰标有925或sterling silver，这表示银饰是纯度为92.5%的银。\n- 考虑女朋友的喜好和风格，选择她会喜欢并经常佩戴的款式。\n- 注意银饰的保养，避免与硬物碰撞或接触汗水、化学品等，以保持银饰的光泽。\n\n希望这些建议能帮助你为女朋友挑选到一款合适的银饰！' response_metadata={'token_usage': {'completion_tokens': 394, 'prompt_tokens': 21, 'total_tokens': 415}, 'model_name': 'glm-4', 'finish_reason': 'stop'} id='run-680e127b-38e1-49ef-a3fb-3e8909f2af88-0'
content='中秋节是一个传统的节日，送银饰给母亲作为礼物是个非常贴心的选择，既体现了孝心也具有美好的寓意。以下是一些建议：\n\n1. **银项链**：可以选择带有福字或者平安扣的银项链，这些传统图案寓意着健康和平安。\n\n2. **银手镯**：银手镯是经典的银饰礼物，可以选择雕刻有牡丹、

### 1.4 多轮对话（记忆模式）
使用 ConversationBufferMemory ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [4]:
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import os

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

ZHIPUAI_API_KEY = os.environ["ZHIPUAI_API_KEY"]
llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "想给女朋友买一款有什么推荐不"
result = qa({"question": question})
print(result['answer'])

question = "第二款好像不错，你详细介绍介绍"
result = qa({"question": question})
print(result['answer'])


h:\exploit_python\environment\anaconda\envs\sml\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


根据提供的信息，以下是几款适合作为女朋友礼物的手镯推荐：

1. **足银999.9古法珐琅彩兰花纯银手镯开口小众款**
   - 价格：36900元
   - 特色：珐琅彩、古法工艺、兰花图案
   - ![足银999.9古法珐琅彩兰花纯银手镯](https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/b1177d19112d909ac2725bbad6f212ecfe3eeffe7bb4d1892536e4138f946941.jpg)

2. **足银999碎冰冰三面纯银手镯实心闭口手镯**
   - 价格：25900元
   - 特色：碎冰冰设计，实心闭口
   - ![足银999碎冰冰三面纯银手镯](https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/b685f541dd19742beb2e3dc014ffc41d14e2771023424d85b9b0510393fa01ec.jpg)

3. **繁花似锦银手镯女纯银9999足银镯子**
   - 价格：34900元
   - 特色：繁花似锦图案，推拉设计
   - ![繁花似锦银手镯女纯银9999足银镯子](https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/52877ffff30ee3716fb65ed9d09e783d7decde3348be42b84ba5480c3c1e0fdd.jpg)

4. **足银999.9古法莫比乌斯扭纹纯银手镯开口小众款**
   - 价格：27900元
   - 特色：莫比乌斯扭纹，开口设计
   - ![足银999.9古法莫比乌斯扭纹纯银手镯](https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/5cd34aabc2416065ae9a67502f453fc69f5d34d0a9324f9d583d073e5d5d2b7e.jpg)

以上手镯都是纯银制品，具有精美的设计和一定的文化内涵，非常适合作为礼物

In [5]:
result

{'question': '第二款好像不错，你详细介绍介绍',
 'chat_history': [HumanMessage(content='想给女朋友买一款有什么推荐不'),
  AIMessage(content='根据提供的信息，以下是几款适合作为女朋友礼物的手镯推荐：\n\n1. **足银999.9古法珐琅彩兰花纯银手镯开口小众款**\n   - 价格：36900元\n   - 特色：珐琅彩、古法工艺、兰花图案\n   - ![足银999.9古法珐琅彩兰花纯银手镯](https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/b1177d19112d909ac2725bbad6f212ecfe3eeffe7bb4d1892536e4138f946941.jpg)\n\n2. **足银999碎冰冰三面纯银手镯实心闭口手镯**\n   - 价格：25900元\n   - 特色：碎冰冰设计，实心闭口\n   - ![足银999碎冰冰三面纯银手镯](https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/b685f541dd19742beb2e3dc014ffc41d14e2771023424d85b9b0510393fa01ec.jpg)\n\n3. **繁花似锦银手镯女纯银9999足银镯子**\n   - 价格：34900元\n   - 特色：繁花似锦图案，推拉设计\n   - ![繁花似锦银手镯女纯银9999足银镯子](https://yinshi-1309445959.cos.ap-beijing.myqcloud.com/yinshi-1309445959/52877ffff30ee3716fb65ed9d09e783d7decde3348be42b84ba5480c3c1e0fdd.jpg)\n\n4. **足银999.9古法莫比乌斯扭纹纯银手镯开口小众款**\n   - 价格：27900元\n   - 特色：莫比乌斯扭纹，开口设计\n   - ![足银999.9古法莫比乌斯扭纹纯银手镯](https://yinshi-1309445959.cos.ap-beijing.myqclo

### 1.5官方实现多轮对话

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")
    
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import os

from langchain_community.chat_models import ChatZhipuAI
ZHIPUAI_API_KEY = os.environ["ZHIPUAI_API_KEY"]
llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)
llm.invoke("请你自我介绍一下自己！")
